# Création d'une base de référence d'agregats

Différents agrégats sont nécessaires pour :
- vérifier la cohérence de nos calculs
- recaller nos résultats de calculs

Dans ce deuxième cas il nous faut :
- calculer le résultat de notre modèle
- calculer un ratio par rapport à l'agrégat
- sauvegarder ce ratio pour l'appliquer en sortie de model

Les agrégats proviennent de deux sources:
- Des calculs que nous faisons sur nos bases
- Des chiffres qui nous sont founis par d'autres administrations tel que l'ACOSS/URSSAF ou la Sécurité Social.

Cette librairie couvre toutes ces utilisations. Le résultat est un fichier YAML qui contient tous les agrégats sous la forme :
- Année
  - Nom de la variable
    - Source de données
      - L'agrégat
      - Son type
      - Le facteur de correction

Dans un premier temps la librairie va charger les fichiers CSV contenant les agregats extraits des bases auxquelles nous avons accès.

Puis on construit une structure de données pour lister les agrégats supplémentaires et les facteurs de corrections à calculer.

Puis nous passons en revue cette structure de données pour effectuer les instructions qu'elle contient.

Enfin nous sauvegardons le YAML sur un dossier partagé.

Note pour le PLF 2023:
- Mettre à jour les agrégats 2021 dans la partie "Set external values"
- Ajouter les agrégats 2022 dans la partie "Set external values"
- Relancer le notebook pour calculer le nouveau fichier
- Vérifier son fonctionnement en intégration
- Si tout est OK, écraser le fichier /mnt/data-out/leximpact/agregats-PROD.yml.

In [1]:
# default_exp aggregates_build

In [2]:
# Activate multi-output in notebook
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# export
import unittest
from typing import List, Optional, Union

import pandas as pd
from pydantic import BaseModel

from leximpact_socio_fisca_simu_etat.config import Configuration
from leximpact_socio_fisca_simu_etat.csg_simu import compute_reform
from leximpact_socio_fisca_simu_etat.schema import ReformeSocioFiscale

config = Configuration()
tc = unittest.TestCase()

[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:40] Connecting to Redis 10.0.0.131:6377


# Construction des agrégats

# Agrégats de fichiers de références

On lit le fichier CSV venant de POTE et on en fait un YAML.

In [4]:
# export
pd.set_option("display.max_colwidth", 80)
pd.options.display.float_format = "{:,.7f}".format

liste_des_variables_csg_2018 = pd.read_csv(
    config.get("CALIB") + "agregats_des_variables_csg-POTE_2018.csv"
)
liste_des_variables_csg_2019 = pd.read_csv(
    config.get("CALIB") + "agregats_des_variables_csg-POTE_2019.csv"
)


pote = {"2019": liste_des_variables_csg_2019, "2018": liste_des_variables_csg_2018}

In [5]:
liste_des_variables_csg_2019.tail(5)

,name,nb_line,lenzero,sum,mean,pct_zero
36,assiette_csg_revenus_capital,39264696,23433471,77579979386,1975,59.6807651
37,retraites,39264696,26424864,307254581479,7825,67.2992960
38,pre_retraites_etranger,39264696,39264583,1511197,0,99.9997122
39,chomage_et_indemnites,39264696,33630886,35110527952,894,85.6517162
40,rev_salaire,39264696,16248529,650855163531,16576,41.3820318


In [6]:
# export


def get_aggregats_from_row(row, year):
    """
    Parcours les lignes du dataframe et enregistre les agrégats dans le dictionnaire.
    ::row:: une ligne de dataframe
    ::year:: year of the data
    """
    global agregats
    if agregats.get(year) is None:
        agregats[year] = {}
    agregats[year][row["name"]] = {}
    source = "POTE"  # TODO: move it in a function parameter
    agregats[year][row["name"]][source] = {}
    agregats[year][row["name"]][source]["sum"] = row["sum"]
    agregats[year][row["name"]][source]["lenzero"] = row["lenzero"]
    agregats[year][row["name"]][source]["pct_zero"] = round(row["pct_zero"], 2)
    agregats[year][row["name"]][source]["mean"] = row["mean"]

In [7]:
# export

agregats = {}

for year, df in pote.items():
    _ = df.apply(get_aggregats_from_row, args=[year], axis=1)

In [8]:
# agregats

In [9]:
# export


def get_aggregate_from_df(df, variable_name):
    """
    Permet de récupérer un agrégat de type somme dans le dataframe
    ::df:: Dataframe à lire
    ::variable_name:: Nom de la variable
    """
    return df.loc[df["name"] == variable_name, "sum"].iloc[0]

In [10]:
tc.assertEqual(
    get_aggregate_from_df(liste_des_variables_csg_2019, "assiette_csg_revenus_capital"),
    77_579_979_386,
)

In [11]:
# export


def nested_set(dic, keys, value):
    """
    Allow to set a value in a nested dictionary
    ::dic:: dic to update
    ::keys:: path to value
    ::value:: value to set
    """
    for key in keys[:-1]:
        if dic.get("key"):
            dic = dic.get("key")
        else:
            dic = dic.setdefault(key, {})
    dic[keys[-1]] = value

# Agregats ne venants pas de fichiers

Ces agrégats proviennent de source externe comme des PDF, on les saisie à la main ici.

## Set external values

Ici nous indiquons quelles sont les valeurs des agrégats externes mais également les facteurs de correction à calculer.

In [12]:
# export


prepare_actions = [
    {
        "agg_name": "csg_salaire",
        "agg_source": "ccss",
        "2020": (55_881 + 16_747) * 1e6,
        "2021": (58_807 + 17_216) * 1e6,
        "of_variables": ["csg_imposable_salaire", "csg_deductible_salaire"],
    },
    {
        "agg_name": "csg_retraite",
        "agg_source": "ccss",
        "2020": 20_944_000_000,
        "2021": 21_291_000_000,
        "of_variables": ["csg_imposable_retraite", "csg_deductible_retraite"],
    },
    {
        "agg_name": "csg_chomage",
        "agg_source": "ccss",
        "2020": 1_623_000_000,
        "2021": 1_037_000_000,
        # "of_variables": ["csg_imposable_chomage", "csg_deductible_chomage"]
    },
    {
        "agg_name": "csg_revenus_capital",
        "agg_source": "ccss",
        "2020": 12_559_000_000,
        "2021": 12_344_000_000,
        # "of_variables": ["csg_revenus_capital"]
    },
    {
        "agg_name": "csg_brute",
        "agg_source": "ccss",
        "2020": 121_735_000_000,
        "2021": 127_252_000_000,
        # "of_variables": ["csg"]
    },
    {
        "agg_name": "crds_salaire_prive",
        "agg_source": "acoss",
        "2019": 3_219_000_000,
        "2020": 3_050_000_000,
        "2021": 3_216_000_000,
    },
    {
        "agg_name": "crds_salaire_public_non_titulaire",
        "agg_source": "acoss",
        "2019": 178_000_000,
        "2020": 185_000_000,
        "2021": 196_000_000,
    },
    {
        "agg_name": "crds_salaire_public_titulaire",
        "agg_source": "acoss",
        "2019": 720_000_000,
        "2020": 724_000_000,
        "2021": 744_000_000,
    },
    # CRDS Activité
    # 7.6 * 1_000_000_000 total CRDS 2019 - CRDS Salaire OF 3,947,173,837 € sans recalage
    # Source de l'agrégat : Mail de l'Acoss du 19 octobre 2021
    {
        "agg_name": "crds_salaire",
        "agg_source": "acoss",
        "2019": 4_117_000_000,
        "2020": 3_959_000_000,
        "2021": 4_156_000_000,
        "of_variables": ["crds_salaire"],
    },
    #     {  #  ! FAUX
    #         "agg_name": "crds_retraite",
    #         "agg_source": "acoss",
    #         "2020": 20_944_000_000,
    #         "2021": 21_291_000_000,
    #     },
    #     {  #  ! FAUX
    #         "agg_name": "crds_chomage",
    #         "agg_source": "acoss",
    #         "2020": 208_734_875, #1_623_000_000
    #         "2021": 200_000_000, # 1_037_000_000
    #     },
    #     {  #  ! FAUX
    #         "agg_name": "crds_revenus_capital",
    #         "agg_source": "acoss",
    #         "2020": 12_559_000_000,
    #         "2021": 12_344_000_000,
    #     },
]

## Prepare data structure

This is an intermediate data structure to store:
- List of Actions
 - Action containing Aggregate and optionaly Simu
  - Aggregate containing metadata about the aggregate
  - Simu containing informations on the simulation to run to compute correction factor

In [13]:
# export


class Aggregate(BaseModel):
    """
    contain metadata about the aggregate
    """

    agg_year: str
    agg_name: str
    agg_source: str
    agg_type: Optional[str] = "sum"
    agg_value: Optional[int] = None
    ux_name: Optional[str] = None


class Simu(BaseModel):
    """
    contain informations on the simulation to run to compute correction factor
    """

    of_year: str
    of_variables: List[str]


class Action(BaseModel):
    """
    contain Aggregate and optionaly Simu
    """

    agg: Aggregate
    simu: Optional[Simu]


class Actions(BaseModel):
    actions: List[Action]

## Build computing list

On utilise la structure de donnée intermédiaire pour créer une autre structure de données plus exhaustive utilisant les BaseModel ci-dessus.

In [14]:
# export
to_process = Actions(actions=[])

for a in prepare_actions:
    if a.get("2019"):
        agg_2019 = Aggregate(
            agg_year="2019",
            agg_name=a["agg_name"],
            agg_source=a["agg_source"],
            agg_value=a["2019"],
        )
        to_process.actions.append(Action(agg=agg_2019))
    agg_2020 = Aggregate(
        agg_year="2020",
        agg_name=a["agg_name"],
        agg_source=a["agg_source"],
        agg_value=a["2020"],
    )
    agg_2021 = Aggregate(
        agg_year="2021",
        agg_name=a["agg_name"],
        agg_source=a["agg_source"],
        agg_value=a["2021"],
    )
    # Factor to compute
    if a.get("of_variables"):
        factor_2021 = Simu(of_year="2021", of_variables=a["of_variables"])
        factor_2022 = Simu(of_year="2022", of_variables=a["of_variables"])
    else:
        factor_2021 = None
        factor_2022 = None
    # Add actions to do
    action = Action(agg=agg_2020)
    to_process.actions.append(action)
    if factor_2021:
        action = Action(agg=agg_2021, simu=factor_2021)
        to_process.actions.append(action)
    action = Action(agg=agg_2021, simu=factor_2022)
    to_process.actions.append(action)

In [15]:
# to_process.dict()

# Calcul du facteur de correction d'une année sur l'autre

Source https://www.securite-sociale.fr/files/live/sites/SSFR/files/medias/CCSS/2021/RAPPORT%20CCSS%20JUIN%202021.pdf page 47

In [16]:
# export


def get_factor(annee_de_calcul: str, variables: Union[List, str], aggregate: int):
    """
    Call Open Fisca to run a simulation and return the correction factor
    ::annee_de_calcul:: Year of computation
    ::variables:: a variable or a list of variables to get as output of the simulation
    ::aggregate:: The value of the aggregate to compute the correction factor
    ::return:: correction factor
    """
    variables = [variables] if isinstance(variables, str) else variables
    reform = ReformeSocioFiscale(
        base=annee_de_calcul,
        amendement={},
        output_variables=variables,
    )
    # Compute the value with Open Fisca
    resultat, errors = compute_reform(reform, annee_de_calcul, ignore_recallage=True)
    # Check there is no error
    tc.assertEqual(errors, [])
    of_result = 0
    for v in variables:
        of_result += resultat.state_budget[v]
    # Print an output
    print(
        f"Montant en sortie OF pour {annee_de_calcul} pour la somme de {variables} : {of_result:,.0f}, montant attendu : {aggregate:,.0f}"
    )
    factor = float(aggregate / abs(of_result))
    return factor

In [17]:
factor = get_factor("2020", "csg_imposable_salaire", 16_906_202_821)
tc.assertAlmostEqual(factor, 1)
factor = get_factor(
    "2020", ["csg_imposable_salaire", "csg_deductible_salaire"], 64_807_110_405
)
tc.assertAlmostEqual(factor, 1)

[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:43] reformParameters : base=2020 plf=None amendement={} output_variables=['csg_imposable_salaire'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:43] No cache for 33e416083fa73b71bfe370b14d3cd50df36eec4fa048cd298100908c2020, compute it.
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:46] reformParameters : base=2020 plf=None amendement={} output_variables=['csg_imposable_salaire', 'csg_deductible_salaire'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:46] No cache for 1e81da669513c1c34a989378a6962421c951fdf0e33ec0f31d275ca32020, compute it.


Montant en sortie OF pour 2020 pour la somme de ['csg_imposable_salaire'] : 16,906,202,821, montant attendu : 16,906,202,821
Montant en sortie OF pour 2020 pour la somme de ['csg_imposable_salaire', 'csg_deductible_salaire'] : 64,807,110,405, montant attendu : 64,807,110,405


In [18]:
# export


def process_actions(list_to_process: Actions):
    """
    Execute the actions in the List of Action : store an aggregate value and/or compute a correction factor
    ::list_to_process:: Instance of Actions
    ::return:: Nothing, it update the global variable agregats
    """
    global agregats
    for a in list_to_process.actions:
        year = a.agg.agg_year
        of_year = a.simu.of_year if a.simu else None
        agg_name = a.agg.agg_name
        agg_source = a.agg.agg_source
        print("Processing", agg_name)
        # If we have a value, we set it
        if a.agg.agg_value:
            nested_set(
                agregats,
                [year, agg_name, agg_source, a.agg.agg_type],
                a.agg.agg_value,
            )
        # Get the aggregate value
        agg = agregats[year][agg_name][agg_source][a.agg.agg_type]
        # Compute the correction factor if needed
        if of_year:
            factor = get_factor(of_year, a.simu.of_variables, agg)
            # Check the precision of the factor
            tc.assertGreater(factor, 0.8, msg=f"for {a.agg.agg_name}")
            tc.assertLess(factor, 1.2, msg=f"for {a.agg.agg_name}")
            if not (0.9 < factor < 1.1):
                print(
                    f"process_actions WARNING : Factor for {agg_name} is above 10% error, but below 20% error."
                )
            # Set the factor on the agregate
            nested_set(
                agregats,
                [year, agg_name, agg_source, f"factor_to_{of_year}"],
                factor,
            )
            # Set the factor on the OpenFisca variables for the ERFS data for the ERFS year
            for v in a.simu.of_variables:
                nested_set(
                    agregats,
                    [config.get("YEAR_ERFS"), v, "ERFS", f"factor_to_{of_year}"],
                    factor,
                )

In [19]:
process_actions(to_process)

[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:51] reformParameters : base=2021 plf=None amendement={} output_variables=['csg_imposable_salaire', 'csg_deductible_salaire'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:51] Cache found for reformParameters 3ab448dddf78cc47e3c09e096ac8b3670fb85573c7e9052371f369fe2021, return it.
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:51] reformParameters : base=2022 plf=None amendement={} output_variables=['csg_imposable_salaire', 'csg_deductible_salaire'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:51] Cache found for reformParameters 39b99142b3cb927bdc04d2472fcc9227c612aa46a6dba8d255583fb72022, return it.
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:51] reformParameters : base=2021 plf=None amendement={} output_variables=['csg_imposable_retraite', 'csg_deductible_retraite'] quantile_nb=0 quanti

Processing csg_salaire
Processing csg_salaire
Montant en sortie OF pour 2021 pour la somme de ['csg_imposable_salaire', 'csg_deductible_salaire'] : 64,807,110,405, montant attendu : 76,023,000,000
process_actions WARNING : Factor for csg_salaire is above 10% error, but below 20% error.
Processing csg_salaire
Montant en sortie OF pour 2022 pour la somme de ['csg_imposable_salaire', 'csg_deductible_salaire'] : 64,807,110,405, montant attendu : 76,023,000,000
process_actions WARNING : Factor for csg_salaire is above 10% error, but below 20% error.
Processing csg_retraite
Processing csg_retraite


[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:58] reformParameters : base=2022 plf=None amendement={} output_variables=['csg_imposable_retraite', 'csg_deductible_retraite'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:06:58] No cache for 44d6b8a7a97ace0b0d312f19e29949aca16667fdcfef292470c620832022, compute it.


Montant en sortie OF pour 2021 pour la somme de ['csg_imposable_retraite', 'csg_deductible_retraite'] : 17,988,795,384, montant attendu : 21,291,000,000
process_actions WARNING : Factor for csg_retraite is above 10% error, but below 20% error.
Processing csg_retraite


[leximpact_socio-fisca-simu-etat DEBUG @ 19:07:04] reformParameters : base=2021 plf=None amendement={} output_variables=['crds_salaire'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:07:04] No cache for 9ec634e1cc1d4c297afdede9fb2d3b7c6e613fd0de74816a964fba6c2021, compute it.


Montant en sortie OF pour 2022 pour la somme de ['csg_imposable_retraite', 'csg_deductible_retraite'] : 17,988,795,384, montant attendu : 21,291,000,000
process_actions WARNING : Factor for csg_retraite is above 10% error, but below 20% error.
Processing csg_chomage
Processing csg_chomage
Processing csg_revenus_capital
Processing csg_revenus_capital
Processing csg_brute
Processing csg_brute
Processing crds_salaire_prive
Processing crds_salaire_prive
Processing crds_salaire_prive
Processing crds_salaire_public_non_titulaire
Processing crds_salaire_public_non_titulaire
Processing crds_salaire_public_non_titulaire
Processing crds_salaire_public_titulaire
Processing crds_salaire_public_titulaire
Processing crds_salaire_public_titulaire
Processing crds_salaire
Processing crds_salaire
Processing crds_salaire


[leximpact_socio-fisca-simu-etat DEBUG @ 19:07:07] reformParameters : base=2022 plf=None amendement={} output_variables=['crds_salaire'] quantile_nb=0 quantile_base_variable=None quantile_compare_variables=None
[leximpact_socio-fisca-simu-etat DEBUG @ 19:07:07] No cache for b9fcdb15668bcc810c1f59ca6a701dfc573962f94a055f86278b5bff2022, compute it.


Montant en sortie OF pour 2021 pour la somme de ['crds_salaire'] : 3,522,125,567, montant attendu : 4,156,000,000
process_actions WARNING : Factor for crds_salaire is above 10% error, but below 20% error.
Processing crds_salaire
Montant en sortie OF pour 2022 pour la somme de ['crds_salaire'] : 3,522,125,567, montant attendu : 4,156,000,000
process_actions WARNING : Factor for crds_salaire is above 10% error, but below 20% error.


## Enregistrement

Nous avons choisi le format YAML pour sa concision et également car c'est le format retenu par Open Fisca.

In [20]:
import yaml

with open(config.get("AGREGATS"), mode="w") as file:
    _ = file.write(yaml.dump(agregats))

# Affichage des agrégats

In [21]:
!cat {config.get("AGREGATS")}

'2018':
  CICS:
    POTE:
      lenzero: 38487869
      mean: 0
      pct_zero: 100.0
      sum: 2730750
  MNIMQG:
    POTE:
      lenzero: 38455801
      mean: 5
      pct_zero: 99.92
      sum: 194860511
  Z1aj:
    POTE:
      lenzero: 17029792
      mean: 13176
      pct_zero: 44.25
      sum: 507120291949
  Z1ap:
    POTE:
      lenzero: 33765299
      mean: 728
      pct_zero: 87.73
      sum: 28041960914
  Z1as:
    POTE:
      lenzero: 26021434
      mean: 6410
      pct_zero: 67.61
      sum: 246717839530
  Z1bj:
    POTE:
      lenzero: 32136117
      mean: 3641
      pct_zero: 83.5
      sum: 140145784316
  Z1bp:
    POTE:
      lenzero: 37297306
      mean: 174
      pct_zero: 96.91
      sum: 6726148004
  Z1bs:
    POTE:
      lenzero: 34397940
      mean: 1378
      pct_zero: 89.37
      sum: 53036816248
  Z1cj:
    POTE:
      lenzero: 37955032
      mean: 47
      pct_zero: 98.62
      sum: 1812139706
  Z1cw:
    POTE:
      lenzero: 38229992
      mean: 12
      pct_ze

In [23]:
# If everything work, copy it to production
#!cp /mnt/data-out/leximpact/agregats.yml /mnt/data-out/leximpact/agregats-PROD.yml